In [1]:
from __future__ import print_function
import torch
from torch.autograd import Variable, Function
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import scipy.io as sio
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from minimax_entropy import MinimaxEntropyEstimator

In [8]:
entro = MinimaxEntropyEstimator('poly_coeff_entro.mat', gpu=True)

# Training settings
batch_size = 64
test_batch_size = 1000
epochs = 1
lr = 0.001
momentum = 0.5
random_seed = 1
log_interval = 10
p = 0
classes = 10

xs = []
targets = []
one_hots = []

In [53]:
torch.cuda.manual_seed(random_seed)

kwargs = {'num_workers': 1, 'pin_memory': True}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)

In [54]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=p, training=self.training)
        x = self.fc2(x) 
        x = F.softmax(x)
        return x

In [65]:
model = Net()
model.cuda()

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

def train(epoch):
    model.train()
    
    target_onehot = Variable(torch.FloatTensor(batch_size, 10))
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        
        target_onehot.data.zero_()
        inds = target.cpu().data.numpy()
        for i in range(batch_size):
            target_onehot[i, inds[i]] = 1.
        print(target_onehot)
        if batch_idx == 1:
            break
        output = model(data).double()

        loss = Variable(torch.zeros(1)).double().cuda()
        for i in range(batch_size):
            loss += entro.cross_entro_loss(output[i], target_onehot[i])
        loss.backward()
        
        if batch_idx == 10:
            break
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
            
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

for epoch in range(1, epochs + 1):
    train(epoch)
    test()

Variable containing:
    0     0     0     0     0     0     1     0     0     0
    0     1     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     1
    0     0     0     0     0     0     0     0     0     1
    0     0     0     0     0     0     1     0     0     0
    1     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     1     0     0     0
    0     0     0     0     0     0     1     0     0     0
    0     0     0     0     0     0     1     0     0     0
    0     0     1     0     0     0     0     0     0     0
    1     0     0     0     0     0     0     0     0     0
    0     1     0     0     0     0     0     0     0     0
    0     1     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     1     0     0
    0     0     0     1     0     0     0     0     0     0
    1     0     0     0     0     0     0     0     0     0
    0     1     0  

In [ ]:
print(one_hots[0][0])
print(targets[0][0])

In [81]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py

In [88]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

# Training settings
batch_size = 64
test_batch_size = 1000
epochs = 2
lr = 0.01
momentum = 0.5
random_seed = 1
log_interval = 10

torch.cuda.manual_seed(random_seed)


kwargs = {'num_workers': 1, 'pin_memory': True}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

model = Net()

model.cuda()

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):

        data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:

        data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.316665
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.310470
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.323934
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.286976
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.279107
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.235492
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.263011
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.204009
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.172151
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.127815
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.043841
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.062152
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.985518
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.018420
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.778770
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.708917
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.781749
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.593200
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.565929
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.467147
Train Epoch: 1 [12800/60000 (